In [1]:
import numpy as np
from gymnasium.spaces import Dict, Box
from ray.rllib.policy.policy import PolicySpec

from envs.rllib import DummyEnvironment, NavigationEnvironment
from examples.agents.policies import MPSTD3Policy
from examples.restrictors.navigation_restrictor import NavigationRestrictor
from src.spaces.interval_union import IntervalUnion
from src.utils import test_environment
from src.wrappers.rllib import UniformlyRestrictedEnvironment

Instructions for updating:
experimental_compile is deprecated, use jit_compile instead
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


/Users/tim/opt/miniconda3/envs/hicss-2024/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/Users/tim/opt/miniconda3/envs/hicss-2024/lib/python3.10/site-packages/tensorflow_probability/python/internal/backend/numpy/dtype.py:82: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bool = np.bool  # pylint: disable=redefined-builtin
/Users/tim/opt/miniconda3/envs/hicss-2024/lib/python3.10/site-packages/tensorflow_probability/python/internal/backend/numpy/dtype.py:112: DeprecationWarning: `np.str` is a dep

In [2]:
test_environment(DummyEnvironment)

NameError: name 'gym' is not defined

In [ ]:
env_config = {'action_space': gym.spaces.Box(0.0, 1.0)}

test_environment(UniformlyRestrictedEnvironment(
    {'env': DummyEnvironment, 'env_config': env_config, 'governance_action_space': gym.spaces.Box(0.0, 1.0)}))

In [2]:
env_config = {
    'STEPS_PER_EPISODE': 4,
    'ACTION_RANGE': 220,
    'DT': 1.0,
    'REWARD': {
        'TIMESTEP_PENALTY_COEFFICIENT': 0.05,
        'REWARD_COEFFICIENT': 5.0,
        'GOAL': 50,
        'COLLISION': -20.0
    },
    'MAP': {
        'HEIGHT': 15.0,
        'WIDTH': 15.0,
        'AGENT': {'x': 1.0, 'y': 1.0, 'angle': 90.0, 'step_size': 1.0, 'radius': 0.4},
        'GOAL': {'x': 12.0, 'y': 12.0, 'radius': 0.5}
    }
}

governance_config = {
    'COUNT': 3,
    'POSITION_COVARIANCE': [[8.0, 0.0],
                            [0.0, 8.0]],
    'MEAN_SIZE': 1.0,
    'VARIANCE_SIZE': 0.25,
    'RANGE_SIZE': 0.75,
    'START_SEED': 50,
    'SAFETY_ANGLE': 6
}

governance_observation_space = Dict({'location': Box(0.0, 20.0, shape=(2,)),
                                                'perspective': Box(0.0, 360.0, shape=(1,)),
                                                'map': Box(0.0, 20.0, shape=(2,)),
                                                'step': Box(0.0, 40.0, shape=(1,)),
                                                'step_radius': Box(0.0, 5.0, shape=(1,)),
                                                'step_size': Box(0.0, 5.0, shape=(1,)),
                                                'action_range': Box(0.0, 360.0, shape=(1,)),
                                                'dt': Box(0.0, 5.0, shape=(1,))})

model_config = {

}


def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    print(agent_id)
    return agent_id


run_config = {
    'env': UniformlyRestrictedEnvironment,
    'env_config': env_config,
    'multiagent': {
        'policies':
            {
                'agent': PolicySpec(
                    policy_class=MPSTD3Policy,
                    observation_space=Dict({
                        'observation': Dict({
                            'location': Box(low=0.0, high=15.0, shape=(2,), dtype=np.float32),
                            'perspective': Box(low=0.0, high=360.0, shape=(1,), dtype=np.float32),
                            'target_angle': Box(low=0.0, high=360.0, shape=(1,), dtype=np.float32),
                            'target_distance': Box(low=0.0, high=50.0, shape=(1,), dtype=np.float32),
                            'current_step': Box(low=0.0, high=40.0, shape=(1,), dtype=np.float32)
                        }),
                        'allowed_actions': IntervalUnion(-110.0, 110.0)
                    }),
                    action_space=Box(low=-110.0, high=110.0, shape=(1,), dtype=np.float32),
                    config=model_config
                ),
                'gov': PolicySpec(
                    policy_class=NavigationRestrictor,
                    observation_space=governance_observation_space,
                    action_space=IntervalUnion(-110.0, 110.0),
                    config=governance_config
                )
            },
        'policy_mapping_fn': policy_mapping_fn,
        'policies_to_train': ['agent']
    }
}


def governance_observation_fn(wrapper):
    return {'location': np.array([wrapper.env.agent.x, wrapper.env.agent.y], dtype=np.float32),
            'perspective': np.array([wrapper.env.agent.perspective], dtype=np.float32),
            'map': np.array([wrapper.env.HEIGHT, wrapper.env.WIDTH], dtype=np.float32),
            'step': np.array([wrapper.env.current_step], dtype=np.float32),
            'step_radius': np.array([wrapper.env.agent.radius], dtype=np.float32),
            'step_size': np.array([wrapper.env.agent.step_size], dtype=np.float32),
            'action_range': np.array([wrapper.env.ACTION_RANGE], dtype=np.float32),
            'dt': np.array([wrapper.env.DT], dtype=np.float32)}


def governance_reward_fn(wrapper):
    return -sum(wrapper.rewards.values())


test_environment(UniformlyRestrictedEnvironment({'env': NavigationEnvironment, 'env_config': env_config,
                                                 'governance_action_space': Box(low=0.0, high=1.0, shape=(2,)),
                                                 'governance_observation_space': governance_observation_space,
                                                 'governance_observation_fn': governance_observation_fn,
                                                 'governance_reward_fn': governance_reward_fn}),
                 NavigationRestrictor(governance_observation_space,
                                      IntervalUnion(-110.0, 110.0),
                                      governance_config))

hi
Dict('action_range': Box(0.0, 360.0, (1,), float32), 'dt': Box(0.0, 5.0, (1,), float32), 'location': Box(0.0, 20.0, (2,), float32), 'map': Box(0.0, 20.0, (2,), float32), 'perspective': Box(0.0, 360.0, (1,), float32), 'step': Box(0.0, 40.0, (1,), float32), 'step_radius': Box(0.0, 5.0, (1,), float32), 'step_size': Box(0.0, 5.0, (1,), float32))
Testing <class 'src.wrappers.rllib.UniformlyRestrictedEnvironment'>...
Reset: {'gov': {'location': array([1., 1.], dtype=float32), 'perspective': array([90.], dtype=float32), 'map': array([15., 15.], dtype=float32), 'step': array([0.], dtype=float32), 'step_radius': array([0.4], dtype=float32), 'step_size': array([1.], dtype=float32), 'action_range': array([220.], dtype=float32), 'dt': array([1.], dtype=float32)}}
L


AttributeError: 'list' object has no attribute 'shape'

In [3]:
from ray.rllib.utils.spaces.repeated import Repeated
import ray

ray.init()
analysis = ray.tune.run('DDPG',
                        config={
                            'env': UniformlyRestrictedEnvironment,
                            'env_config': {
                                'env': NavigationEnvironment, 'env_config': env_config,
                                'governance_action_space': Repeated(Box(low=-180.0, high=180.0, shape=(2,)), max_len=20),
                                'governance_observation_space': governance_observation_space,
                                'governance_observation_fn': governance_observation_fn,
                                'governance_reward_fn': governance_reward_fn,
                            },
                            'multiagent': {
                                'policies':
                                    {
                                        'agent': PolicySpec(
                                            policy_class=MPSTD3Policy,
                                            observation_space=Dict({
                                                'observation': Dict({
                                                    'location': Box(low=0.0, high=15.0, shape=(2,), dtype=np.float32),
                                                    'perspective': Box(low=0.0, high=360.0, shape=(1,),
                                                                       dtype=np.float32),
                                                    'target_angle': Box(low=0.0, high=360.0, shape=(1,),
                                                                        dtype=np.float32),
                                                    'target_distance': Box(low=0.0, high=50.0, shape=(1,),
                                                                           dtype=np.float32),
                                                    'current_step': Box(low=0.0, high=40.0, shape=(1,),
                                                                        dtype=np.float32)
                                                }),
                                                'allowed_actions': Repeated(Box(low=-180.0, high=180.0, shape=(2,),
                                                                                dtype=np.float32), max_len=20)
                                            }),
                                            action_space=Box(low=-110.0, high=110.0, shape=(1,), dtype=np.float32),
                                            config=model_config
                                        ),
                                        'gov': PolicySpec(
                                            policy_class=NavigationRestrictor,
                                            observation_space=governance_observation_space,
                                            action_space=IntervalUnion(-180.0, 180.0),
                                            config=governance_config
                                        )
                                    },
                                'policy_mapping_fn': policy_mapping_fn,
                                'policies_to_train': ['agent']
                            },
                            'framework': 'torch'
                        },
                        stop={'num_env_steps_sampled': 5000})

2023-05-22 17:41:07,371	INFO worker.py:1625 -- Started a local Ray instance.


(pid=78480) /Users/tim/opt/miniconda3/envs/hicss-2024/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=78480)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=78480) /Users/tim/opt/miniconda3/envs/hicss-2024/lib/python3.10/site-packages/tensorflow_probability/python/internal/backend/numpy/dtype.py:82: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
(pid=78480) Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
(pid=78480)   bool = np.bool  # pylint: disable=redefined-builtin
(pid=78480) /Users/tim/opt/miniconda3/envs/hicss-2024/lib/python3.10/site-packages/tensorflow_probability/python/in

(DDPG pid=78480) (8,)
(DDPG pid=78480) Box(-1.0, 1.0, (47,), float32)
(DDPG pid=78480) (47,)
(DDPG pid=78480) Box(-1.0, 1.0, (47,), float32)
(DDPG pid=78480) (47,)
(DDPG pid=78480) Box(-110.0, 110.0, (1,), float32)
(DDPG pid=78480) (1,)
(DDPG pid=78480) Box(-110.0, 110.0, (1,), float32)
(DDPG pid=78480) (1,)
(DDPG pid=78480) Box(-inf, inf, (), float32)
(DDPG pid=78480) ()
(DDPG pid=78480) Box(-inf, inf, (), float32)
(DDPG pid=78480) ()
(DDPG pid=78480) Box(-inf, inf, (), float32)
(DDPG pid=78480) ()
(DDPG pid=78480) Box(-inf, inf, (), float32)
(DDPG pid=78480) ()
(DDPG pid=78480) Box(-inf, inf, (), float32)
(DDPG pid=78480) ()
(DDPG pid=78480) Box(-inf, inf, (), float32)
(DDPG pid=78480) ()
(DDPG pid=78480) Box(-inf, inf, (), float32)
(DDPG pid=78480) ()
(DDPG pid=78480) Box(-inf, inf, (), float32)
(DDPG pid=78480) ()
(DDPG pid=78480) Box(-inf, inf, (), float32)
(DDPG pid=78480) ()
(DDPG pid=78480) Box(-1.0, 1.0, (47,), float32)
(DDPG pid=78480) (47,)
(DDPG pid=78480) Box(-1.0, 1.0, (4

(DDPG pid=78480) 2023-05-22 17:41:23,036	WARNING util.py:67 -- Install gputil for GPU system monitoring.
2023-05-22 17:41:23,144	ERROR trial_runner.py:1450 -- Trial DDPG_UniformlyRestrictedEnvironment_12f10_00000: Error happened when processing _ExecutorEventType.TRAINING_RESULT.
ray.exceptions.RayTaskError(AssertionError): ray::DDPG.train() (pid=78480, ip=127.0.0.1, repr=DDPG)
  File "/Users/tim/opt/miniconda3/envs/hicss-2024/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 384, in train
    raise skipped from exception_cause(skipped)
  File "/Users/tim/opt/miniconda3/envs/hicss-2024/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 381, in train
    result = self.step()
  File "/Users/tim/opt/miniconda3/envs/hicss-2024/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 792, in step
    results, train_iter_ctx = self._run_one_training_iteration()
  File "/Users/tim/opt/miniconda3/envs/hicss-2024/lib/python3.10/site-packages/ray/r

Trial name,date,hostname,node_ip,pid,timestamp,trial_id
DDPG_UniformlyRestrictedEnvironment_12f10_00000,2023-05-22_17-41-23,vpn-135-057.rz.uni-mannheim.de,127.0.0.1,78480,1684770083,default


(DDPG pid=78480) 2023-05-22 17:41:23,115	WARNING env_runner_v2.py:991 -- Your environment seems to be stepping w/o ever emitting agent observations (agents are never requested to act)!
(DDPG pid=78480) /Users/tim/opt/miniconda3/envs/hicss-2024/lib/python3.10/site-packages/ray/rllib/evaluation/collectors/agent_collector.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
(DDPG pid=78480)   arr = np.array(v)


(DDPG pid=78480) Box(-inf, inf, (), float32)
(DDPG pid=78480) ()
(DDPG pid=78480) {'obs': ViewRequirement(data_col=None, space=Box(-1.0, 1.0, (10,), float32), shift=0, index=None, batch_repeat_value=1, used_for_compute_actions=True, used_for_training=True, shift_arr=array([0])), 'new_obs': ViewRequirement(data_col='obs', space=Box(-1.0, 1.0, (10,), float32), shift=1, index=None, batch_repeat_value=1, used_for_compute_actions=False, used_for_training=True, shift_arr=array([1])), 'actions': ViewRequirement(data_col=None, space=<IntervalUnion>, shift=0, index=None, batch_repeat_value=1, used_for_compute_actions=False, used_for_training=True, shift_arr=array([0])), 'prev_actions': ViewRequirement(data_col='actions', space=<IntervalUnion>, shift=-1, index=None, batch_repeat_value=1, used_for_compute_actions=True, used_for_training=True, shift_arr=array([-1])), 'rewards': ViewRequirement(data_col=None, space=Box(-inf, inf, (), float32), shift=0, index=None, batch_repeat_value=1, used_for_com

TuneError: ('Trials did not complete', [DDPG_UniformlyRestrictedEnvironment_12f10_00000])